In [1]:
import numpy as np
import numpy.random as rd
import numba as nb

# Initial Grid

In [2]:
def spins_generator(N):
    """
    Generates a 2D grid of spins, the X and Y direction are enumerated in the tex file.
    """
    spins = rd.choice(np.array([-1,1]),size = (N,N))
    return spins

# Monte Carlo Energy

In [3]:
@nb.njit
def Hamiltonian_init(grid,J,B):
    E =0
    N = len(grid)
    for X in range(N):
        for Y in range(N):
            E -= B * grid[X,Y]
            E+= 0.25*J*grid[X,Y] * (grid[(X+1)%N,Y] + grid[(X-1)%N,Y])
            E+= 0.25*J*grid[X,Y] * (grid[X,(Y+1)%N] + grid[X,(Y-1)%N])
            E+= 0.25*J*grid[X,Y] * (grid[(X+1)%N,(Y-1)%N] + grid[(X-1)%N,(Y+1)%N])
    return E

In [4]:
@nb.njit
def MCV_step(Nsteps,beta,init,J=1,B=0):
    """
    N:= Number of Mc steps;
    beta:= inverse temperature;
    J = exchange term, set to 1 default
    B = magnetic field set to 0 default.
    """
    
    N  = len(init)
    

    #initial_energy
    E0 = Hamiltonian_init(init,J,B)
    
    #variables to compute mean and variance on the fly:
    E = E0
    varE = 0
    dummy = 0
    E2 = E0*E0
    varE2 = 0
    
    for k in range(Nsteps):
        ## Metropolis step
        X,Y = rd.randint(0,N,2)
        #calculate the total change of energy:
        v  = init[X,Y] * (init[(X+1)%N,Y] + init[(X-1)%N,Y])
        v += init[X,Y] * (init[X,(Y+1)%N] + init[X,(Y-1)%N])
        v += init[X,Y] * (init[(X+1)%N,(Y-1)%N] + init[(X-1)%N,(Y+1)%N])
        
        dE = 2*B*init[X,Y] - J*v
        
        if rd.random() < np.exp(-beta*dE):
            E0 += dE
            init[X,Y] *= -1
        
        dummy = (E0-E) 
        E += dummy/(k+2.)
        
        varE += dummy*(E0-E)/(k+2.) - varE/(k+2.) 
    
    return E/(N*N),np.sqrt(varE)/(N*N)

In [5]:
@nb.njit
def energy_vs_temperature(betas,Nsteps,warmup,init,J,B):
    size = len(init)
    En = np.zeros(len(betas))
    stdEn = np.zeros(len(betas))
    for i,b in enumerate(betas):
        E,stdE = MCV_step(Nsteps,1./b,init,J,B)
        En[i] = E
        stdEn[i] = stdE
    return En,stdEn